# Taken From Pinecone Cookbook 
https://python.langchain.com/en/latest/getting_started/concepts.html

In [1]:
%pip install -qU langchain openai google-search-results wikipedia sqlalchemy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain import OpenAI
import os
key = os.environ.get('OPENAI_API_KEY')
from langchain import OpenAI
llm = OpenAI(
    openai_api_key=key,
    temperature=0
)


In [4]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [7]:
from sqlalchemy import Column, Integer, String, Table, Date, Float
# check if table exists in database and if not, create it
# use inspect 
from sqlalchemy import inspect
inspector = inspect(engine)
got_table = False
for table_name in inspector.get_table_names():
    if table_name == "stocks":
        got_table = True
        break
if not got_table:
    stocks = Table(
        "stocks",
        metadata_obj,
        Column("obs_id", Integer, primary_key=True),
        Column("stock_ticker", String(4), nullable=False),
        Column("price", Float, nullable=False),
        Column("date", Date, nullable=False),    
    )
    print("Table created")



Table created


In [8]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [9]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [10]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [11]:
for obs in observations:
    insert_obs(obs)

In [16]:
%pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.sql_database import SQLDatabase
#from langchain.chains import SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_experimental/sql/base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [18]:
from langchain.agents import Tool

sql_tool = Tool(
    name='Stock DB',
    func=sql_chain.run,
    description="Useful for when you need to answer questions about stocks " \
                "and their prices."
    
)

In [19]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

In [20]:
tools.append(sql_tool)

In [21]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [22]:
result = count_tokens(
    zero_shot_agent, 
    "What is the difference between sum of the stock " +
    "prices for 'ABC' and 'XYZ' on January 3rd and" + 
    " the sum of the same stocks on January the 4th?"
)



> Entering new AgentExecutor chain...
 I need to compare the stock prices of two stocks on two different days
Action: Stock DB
Action Input: Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th

> Entering new SQLDatabaseChain chain...
Stock prices of 'ABC' and 'XYZ' on January 3rd and January 4th
SQLQuery:SELECT stock_ticker, price, date FROM stocks WHERE (stock_ticker = 'ABC' OR stock_ticker = 'XYZ') AND (date = '2023-01-03' OR date = '2023-01-04')
SQLResult: [('ABC', 232.0, '2023-01-03'), ('ABC', 225.0, '2023-01-04'), ('XYZ', 798.0, '2023-01-03'), ('XYZ', 795.0, '2023-01-04')]
Answer:On January 3rd, the price of ABC was 232.0 and the price of XYZ was 798.0. On January 4th, the price of ABC was 225.0 and the price of XYZ was 795.0.
> Finished chain.

Observation: On January 3rd, the price of ABC was 232.0 and the price of XYZ was 798.0. On January 4th, the price of ABC was 225.0 and the price of XYZ was 795.0.
Thought: I need to calculate the difference between the two su

In [23]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Stock DB: Useful for when you need to answer questions about stocks and their prices.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Stock DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Agent Type 2

In [24]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

In [25]:
tools.append(sql_tool)

In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [27]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [28]:
result = count_tokens(
    conversational_agent, 
    "Please provide me the stock prices for ABC on January the 1st"
                     )



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC on January the 1st

> Entering new SQLDatabaseChain chain...
ABC on January the 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'
SQLResult: [(200.0,)]
Answer:The price of ABC on January the 1st was 200.0.
> Finished chain.

Observation: The price of ABC on January the 1st was 200.0.
Thought: Do I need to use a tool? No
AI: The price of ABC on January the 1st was 200.0. Is there anything else I can help you with?

> Finished chain.
Spent a total of 1847 tokens


In [29]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [30]:
result = count_tokens(
    conversational_agent, 
   "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " + 
    "between the same stock prices in January the 4th?"
)




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: The multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th

ValueError: LLMMathChain._evaluate("
(ABC_Jan3 / XYZ_Jan3) * (ABC_Jan4 / XYZ_Jan4)
") raised error: 'ABC_Jan3'. Please try again with a valid numerical expression

# Agent 3

In [29]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [30]:
docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

In [31]:
count_tokens(docstore_agent, "What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was a Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemat

{'input': "What were Archimedes' last words?",
 'output': '"Do not disturb my circles"'}

# Agent 4


In [32]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [33]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin